In [9]:
import pandas as pd
import numpy as np
import skimage

from topact.densetools import density

In [3]:
bin20 = pd.read_csv('../topact-data/kidney-stereoseq/bin20_classifications.csv', 
                    usecols=['x', 'y', 'mouse_ID', 'clusterid_0_6'])

In [4]:
bin20pods = bin20[bin20['clusterid_0_6'].str.match(r'(.)*pods')]

samples = {'control_1': 'ctrl_1',
           'control_2': 'ctrl_2',
           'control_3': 'ctrl_3',
           'control_4': 'ctrl_4',
           '4032_1': 'm32_2', # Intentional -- the labels are mixed up in the CSV .
           '4032_2': 'm32_1', # Can be verified by comparing coordinates.
           '4032_3': 'm32_4',
           '4032_4': 'm32_3',
           '4030_1': 'm30_1',
           '4030_2': 'm30_2'
          }

In [5]:
offsets_csv = pd.read_csv('../topact-data/kidney-stereoseq/offsets.csv')

offsets = {}

for row in offsets_csv.itertuples():
    offsets[row.sample] = (row.xmin, row.ymin)

In [11]:
for sample in samples:
    df = bin20pods[bin20pods.mouse_ID == samples[sample]]
    xmin, xmax, ymin, ymax = df.x.min(), df.x.max(), df.y.min(), df.y.max()
    N, M = xmax - xmin + 1, ymax - ymin + 1
    
    matrix = np.zeros((M//20 + 1,N//20 + 1))
    
    for row in df.itertuples():
        matrix[(row.y - ymin)//20, (row.x-xmin)//20] = 1

    d = density(matrix,1)

    pods = skimage.feature.blob_dog(d, threshold=0.2)[:,:2] * 20 + 10

    col = pods[:,0].copy()
    pods[:,0] = pods[:,1] + xmin
    pods[:,1] = col + ymin
    
    np.savetxt(f'../topact-data/kidney-stereoseq/bin20-gloms/{sample}_gloms.txt', pods, fmt='%1.f')